In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from textwrap import dedent     # tap(들여쓰기)에 상관없이 쓸 수 있게.
from pprint import pprint

In [3]:
from langchain_community.tools import TavilySearchResults

query = "스테이크와 어울리는 와인을 추천해 주세요"

web_search = TavilySearchResults(max_results=2)     # max_result의 갯수까지만 문서를 가져오겠다.

search_results = web_search.invoke(query)           # TavilySearch도 검색 결과기 때문에 invoke() 사용

for result in search_results:
    print(result)
    print('-'*50)

H
--------------------------------------------------
T
--------------------------------------------------
T
--------------------------------------------------
P
--------------------------------------------------
E
--------------------------------------------------
r
--------------------------------------------------
r
--------------------------------------------------
o
--------------------------------------------------
r
--------------------------------------------------
(
--------------------------------------------------
'
--------------------------------------------------
4
--------------------------------------------------
0
--------------------------------------------------
1
--------------------------------------------------
 
--------------------------------------------------
C
--------------------------------------------------
l
--------------------------------------------------
i
--------------------------------------------------
e
--------------------------------------------

In [4]:
print(type(web_search))             # 타입반환

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>


In [5]:
print(web_search.name)              # 풀네임

tavily_search_results_json


In [6]:
print(web_search.description)       # llm을 위해 설명하는 것

A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.


In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

llm_with_tools = llm.bind_tools(tools=[web_search])

In [8]:
# 간단한 쿼리문 입력
query = "안녕하세요"
ai_msg = llm_with_tools.invoke(query)
pprint(ai_msg)
pprint('-'*100)

pprint(ai_msg.content)
pprint('-'*100)

pprint(ai_msg.tool_calls)           # llm이 tool을 호출할지 말지를 판단할 수 있음. 
pprint('-'*100)


AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 81, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'stop', 'logprobs': None}, id='run--0632711a-781d-44de-a18f-a143d6dad537-0', usage_metadata={'input_tokens': 81, 'output_tokens': 11, 'total_tokens': 92})
'----------------------------------------------------------------------------------------------------'
'안녕하세요! 어떻게 도와드릴까요?'
'----------------------------------------------------------------------------------------------------'
[]
'----------------------------------------------------------------------------------------------------'


In [9]:
# 복잡한 쿼리를 입력
query = "스테이크와 어울리는 와인을 추천해 주세요."
ai_msg = llm_with_tools.invoke(query)
pprint(ai_msg)
pprint('-'*100)

pprint(ai_msg.content)              # 컨텐츠를 사용하지 않을 때는 tool을 사용한 경우임. 따라서 llm이 tool을 쓰는데 content를 반환하면 잘못 설계된것
pprint('-'*100)
# tool을 가진 llm에게는 어떤 tool을 호출할지 맡기는 것을 중요하게 봄. [내부에는 복수의 tool 호출 가능하다.]
pprint(ai_msg.tool_calls)
pprint('-'*100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_E636r58T9qprO094EAJCNEZJ', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 92, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7dc9796b-ac0e-4c90-afba-c87d1e011f00-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_E636r58T9qprO094EAJCNEZJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 27, 'total_tokens': 119})
'------------------------------------------------

In [10]:
tool_call = ai_msg.tool_calls[0]
tool_call

{'name': 'tavily_search_results_json',
 'args': {'query': '스테이크와 어울리는 와인 추천'},
 'id': 'call_E636r58T9qprO094EAJCNEZJ',
 'type': 'tool_call'}

In [11]:
# tool call에 대해서 어떻게 생각 하는지 찍어 보기 위함.
tool_message = web_search.invoke(tool_call)

print(tool_message)

content="HTTPError('401 Client Error: Unauthorized for url: https://api.tavily.com/search')" name='tavily_search_results_json' tool_call_id='call_E636r58T9qprO094EAJCNEZJ' artifact={}


In [12]:
pprint(tool_message.content)

("HTTPError('401 Client Error: Unauthorized for url: "
 "https://api.tavily.com/search')")


In [13]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

today= datetime.today().strftime("%Y-%m-%d")
# prompt 생성
prompt = ChatPromptTemplate([
    ("system", f"You are helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder","{message}")             # tools의 호출에 대한 정보 로드
])
# llm과 tool 가져오기
llm = ChatOpenAI(model='gpt-4o-mini')
llm_with_tools = llm.bind_tools(tools=[web_search])
# llm chain 생성
llm_chain = prompt | llm_with_tools
# 함수를 체인화
@chain
def web_search_chain(user_input: str):
    input_ = {"user_input":user_input}
    ai_msg = llm_chain.invoke(input_)
    # ai의 메세지가 여러개 올 수 있기에 batch사용
    tool_msgs = web_search.batch(ai_msg.tool_calls)
    print("tool_msgs: \n", tool_msgs)
    print('-'*100)
    # 결과 반환
    # **변수 -> dictionary로 풀어서 사용 되기 위해 (이후 인자가 dictionary 형태로 오기때문에 unpack 문법을 사용함. )
    return llm_chain.invoke({**input_, "message":[ai_msg, *tool_msgs]})          # message는 placeholder의 (key, value)

response = web_search_chain.invoke("오늘 모엣샹동 샴페인의 원화 가격은 얼마인가요?")
pprint(response.content)

tool_msgs: 
 [ToolMessage(content="HTTPError('401 Client Error: Unauthorized for url: https://api.tavily.com/search')", name='tavily_search_results_json', tool_call_id='call_FrJ1nDX02YRPaw3voirTBSC2', artifact={})]
----------------------------------------------------------------------------------------------------
''


In [14]:
# 사용자 커스텀 도구 정의하기
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from typing import List

@tool       # 커스텀 tool 생성 어노테이션
def search_web(query: str) -> str:
    """Searches the internet for information that does not exist in the database or 
    for the latest information."""
    
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)
    
    formatted_docs = "\n---\n".join([
            f'<Document href="{doc["url"]}"/>\n{doc["content"]}\n</Document>'
            for doc in docs
            ])

    if len(formatted_docs) > 0:
        return formatted_docs

    return "관련 정보를 찾을 수 없습니다."

In [15]:
query = "스테이크와 어울리는 와인을 추천해 주세요."
search_result = search_web.invoke(query)
print(search_result)

TypeError: string indices must be integers, not 'str'

In [ ]:
llm_with_tools = llm.bind_tools(tools=[search_web])

query = "스테이크와 어울리는 와인을 추천해 주세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
pprint('-'*100)

pprint(ai_msg.content)              # 결과가 도출 안 될 수 있음. 웹 검색을 한 정보가 도출 될 때는 빈 값으로 나오게 됨.
pprint('-'*100)

pprint(ai_msg.tool_calls)           # llm이 tool을 호출할지 말지를 판단할 수 있음. 
pprint('-'*100)

In [16]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [ ]:
import re
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크
내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, ...

메뉴 번호: 2
메뉴 이름: 트러플 리조또
내용:
2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조...


In [18]:
# Chroma Vectorstore를 사용하기 위한 준비
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

# Chroma 인덱스 생성
menu_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


검색 결과: 2개
메뉴 번호: 1
메뉴 이름: 시그니처 스테이크

메뉴 번호: 8
메뉴 이름: 안심 스테이크 샐러드



In [19]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# Chroma 인덱스 생성
wine_db = Chroma.from_documents(
    documents=menu_documents, 
    embedding=embeddings_model,   
    collection_name="restaurant_wine",
    persist_directory="./chroma_db",
)

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 샤토 마고 2015
내용:
1. 샤토 마고 2015
   • 가격: ₩450,000
   • 주요 품종: 카베르네 소비뇽, 메를로, 카베르네 프랑, 쁘띠 베르도
   • 설명: 보르도 메독 지역의 프리미엄 ...

메뉴 번호: 2
메뉴 이름: 돔 페리뇽 2012
내용:
2. 돔 페리뇽 2012
   • 가격: ₩380,000
   • 주요 품종: 샤르도네, 피노 누아
   • 설명: 프랑스 샴페인의 대명사로 알려진 프레스티지 큐베입니다. 시트러스...


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


검색 결과: 2개
메뉴 번호: 10
메뉴 이름: 그랜지 2016

메뉴 번호: 9
메뉴 이름: 샤토 디켐 2015



In [ ]:
# 벡터 저장소 로드
menu_db = Chroma(
    embedding_function=embeddings_model,   
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [21]:
@tool
def search_menu(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) >0: return docs
    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

In [22]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# 벡터 저장소 로드
wine_db = Chroma(
   embedding_function=embeddings_model,   
   collection_name="restaurant_wine",
   persist_directory="./chroma_db",
)

@tool
def search_wine(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs
   
   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [23]:
llm_with_tools = llm.bind_tools(tools=[search_menu, search_wine])
query="시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
pprint('-'*100)

pprint(ai_msg.content)              # 결과가 도출 안 될 수 있음. 웹 검색을 한 정보가 도출 될 때는 빈 값으로 나오게 됨.
pprint('-'*100)

pprint(ai_msg.tool_calls)           # llm이 tool을 호출할지 말지를 판단할 수 있음. 
pprint('-'*100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YsxHftGE3iYPkycNBzBFgPIm', 'function': {'arguments': '{"query": "시그니처 스테이크"}', 'name': 'search_menu'}, 'type': 'function'}, {'id': 'call_Uwll2whqJnJx20z31ccY7kcl', 'function': {'arguments': '{"query": "스테이크"}', 'name': 'search_wine'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 137, 'total_tokens': 190, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d87ba716-5b7c-421f-9147-97f6c48be3a2-0', tool_calls=[{'name': 'search_menu', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_YsxHftGE3iYPkycNBzBFgPIm', 'type': 'tool_call'}, {'name': 'search_wine', 'args': {'query

In [24]:
tools = [search_web, search_wine, search_menu]
for tool in tools:
    print(tool.name)

search_web
search_wine
search_menu


In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 3개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)       

        elif tool_call["name"] == "search_wine":
            tool_message = search_wine.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_menu":
            tool_message = search_menu.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
# response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.")
response = restaurant_menu_chain.invoke("전세계에서 가장 비싼 스테이크에 대해 알려 주세요.")

# 응답 출력 
print(response.content)

search_menu: 
{'name': 'search_menu', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_Fwu7Bz2ElHXfdhEwUnG139Hr', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_wine: 
{'name': 'search_wine', 'args': {'query': '스테이크와 어울리는 와인'}, 'id': 'call_wQlFDw9fU4Fki0mqDghQk5L7', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '안심 스테이크 샐러드', 'menu_number': 8, 'source': './data/restaurant_menu.txt'}, page_content='8. 안심 스테이크 샐러드\\n   • 가격: ₩2